In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('STOCK.txt')

In [ ]:
data.head()

In [ ]:
data[10000:].describe()

In [ ]:
data.info()

In [ ]:
data.tail()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data[9500:10000].Open.rolling(1).sum())
plt.title('Data')
plt.grid(True)
plt.show()
#ye_2017_normal = year_2017[:-35]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data[10000:].Open)
plt.title('Data')
plt.grid(True)
plt.show()
#ye_2017_normal = year_2017[:-35]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(30, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data.index[10000:],data[10000:].High.rolling(10).sum(),data.index[10000:],data[10000:].Low.rolling(10).sum())

plt.title('Data')
plt.grid(True)
plt.show()
#ye_2017_normal = year_2017[:-35]

In [ ]:
data['range'] = data.High - data.Low

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data[11650:].range.rolling(15).sum())
plt.title('Data')
plt.grid(True)
plt.show()
#ye_2017_normal = year_2017[:-35]

In [ ]:
copy = list(data[:12073].High.values)
difference = list(data.High[1:] - copy)
difference.append(0)
data['difference'] = difference

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data[10000:].difference.rolling(15).sum())
plt.title('Data')
plt.grid(True)
plt.show()

In [ ]:
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
X = data.High.values
result = adfuller(X[:])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
data.describe()

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series
from matplotlib import pyplot as plt
from scipy.stats.stats import pearsonr 
import datetime
import multiprocessing
import math

#input file, formatted as having 2 columns: position, and value
txn = pd.read_csv('STOCK.txt')
#txn = txn.drop_duplicates()
#txn_original = txn.copy()
txn.drop(['High','Low','Close','Volume','OpenInt'],axis = 1, inplace = True)
txn.columns = ['coord', 'value']
#txn = txn.set_index('coord')

MAX_LAG = 10

LENGTH_GENOME = max(txn.index)

# For each lag, this for loop identifies pairs of loci in the genome with that lag, and saves their 
# expression values in a list. Then it computes the correlations between the 2 lists.

def pearson_correlation(numbers_x, numbers_y):
    x = numbers_x - np.mean(numbers_x)
    y = numbers_y - np.mean(numbers_y)
    return (x * y).sum() / np.sqrt((x**2).sum() * (y**2).sum())

def get_corr(lag):
    exp1 = []
    exp2 = []
    for i in txn.index:
        e1 = txn.value[i]
        if (i + lag) > LENGTH_GENOME:
            break
        if (i + lag) in txn.index:
            e2 = txn.value[i+lag]
            print(e2)
            if  hasattr(e1,"__len__"):
                e1 = e1.iloc[0]
            if  hasattr(e2,"__len__"):
                e2 = e2.iloc[0]
            exp1.append(e1)
            exp2.append(e2)
    return[lag, pearson_correlation(exp1,exp2)]

RES = []
for i in range(MAX_LAG):
    p = get_corr(i)
    RES.append(p)

lags = []
cors = []
for i in range(0, len(RES)):
    lags.append(RES[i][0])
    cors.append(RES[i][1])

# Forming the correlation matrix
def ACF_mat(cors):
    corr_mat = []
    for i in range(len(cors)):
        lag_x = []
        for j in range(MAX_LAG):
            lag_x.append(cors[abs(j-i)])
        corr_mat.append(lag_x)
    return(corr_mat)

cor_mat = ACF_mat(cors)
def AR_Params(ACF_mat):
    beta = np.random.rand(len(ACF_mat))
    beta = np.dot(np.linalg.inv(ACF_mat),ACF_mat[0])
    return(beta)    

beta = AR_Params(cor_mat)
print("This is BETA")
print(beta)


In [ ]:
for i in corr_mat.index:
    print(i)

In [ ]:
def pearson_correlation(numbers_x, numbers_y):
    x = numbers_x - np.mean(numbers_x)
    y = numbers_y - np.mean(numbers_y)
    return (x * y).sum() / np.sqrt((x**2).sum() * (y**2).sum())

def get_cors(lag):
    exp1 = []
    exp2 = []
    LENGTH_GENOME = len(data)
    data_index = [i for i in range(LENGTH_GENOME)]
    for i in range(len(data)):
        e1 = data[i]
        if (i + lag) > LENGTH_GENOME:
            break
        if (i + lag) in data_index:
            e2 = data[i+lag]
            print(e2)
            exp1.append(e1)
            exp2.append(e2)                
        return(pearson_correlation(exp1,exp2))

class AR_MODEL:
    def __init__(self, lag, data):
        self.lag = lag
        self.data = data 
        
    def ACF_mat(self, max_lag):
        cors = []
        for i in range(max_lag):
            cors.append(get_cors(i))
        corr_mat = []
        for i in range(len(cors)):
            lag_x = []
            for j in range(max_lag):
                lag_x.append(cors[abs(j-i)])
            corr_mat.append(lag_x)
        return(corr_mat)
        
    def model_params(self, max_lag, data):
        beta = np.random.rand(max_lag)
        mat = self.ACF_mat(max_lag, data)
        beta = np.dot(np.linalg.inv(mat),mat[0])
        return(beta)
        
        
        
    
        

In [ ]:
model = AR_MODEL(10, data)

In [ ]:
model.ACF_mat(10)

In [ ]:
import numpy as np 

x = np.array([[1,2],[3,4]]) 
y = np.linalg.inv(x) 
print(x) 
print(y) 
print(np.dot(x,y))

In [ ]:
list

In [4]:
import pandas as pd
import numpy as np
from pandas import Series
from matplotlib import pyplot as plt
from scipy.stats.stats import pearsonr 
import datetime
import multiprocessing
import math

#input file, formatted as having 2 columns: position, and value
txn = pd.read_csv('STOCK.txt')
#txn = txn.drop_duplicates()
#txn_original = txn.copy()
txn.drop(['High','Low','Close','Volume','OpenInt'],axis = 1, inplace = True)
txn.columns = ['coord', 'value']
#txn = txn.set_index('coord')

MAX_LAG = 10

LENGTH_GENOME = max(txn.index)

# For each lag, this for loop identifies pairs of loci in the genome with that lag, and saves their 
# expression values in a list. Then it computes the correlations between the 2 lists.

def pearson_correlation(numbers_x, numbers_y):
    x = numbers_x - np.mean(numbers_x)
    y = numbers_y - np.mean(numbers_y)
    return (x * y).sum() / np.sqrt((x**2).sum() * (y**2).sum())

def get_corr(lag):
    exp1 = []
    exp2 = []
    for i in txn.index:
        e1 = txn.value[i]
        if (i + lag) > LENGTH_GENOME:
            break
        if (i + lag) in txn.index:
            e2 = txn.value[i+lag]
            #print(e2)
            if  hasattr(e1,"__len__"):
                e1 = e1.iloc[0]
            if  hasattr(e2,"__len__"):
                e2 = e2.iloc[0]
            exp1.append(e1)
            exp2.append(e2)
    return([lag, pearson_correlation(exp1,exp2)])

RES = []
for i in range(MAX_LAG):
    p = get_corr(i)
    RES.append(p)

lags = []
cors = []
for i in range(0, len(RES)):
    lags.append(RES[i][0])
    cors.append(RES[i][1])

# Forming the correlation matrix
def ACF_mat(cors):
    corr_mat = []
    for i in range(len(cors)):
        lag_x = []
        for j in range(MAX_LAG):
            lag_x.append(cors[abs(j-i)])
        corr_mat.append(lag_x)
    return(corr_mat)

cor_mat = ACF_mat(cors)
print("This is correlation matrix \n")
print(cor_mat)
def AR_Params(ACF_mat):
    beta = np.random.rand(len(ACF_mat))
    beta = np.dot(np.linalg.inv(ACF_mat),ACF_mat[0])
    return(beta)    

beta = AR_Params(cor_mat)
print("This is BETA")
print(beta)


This is correlation matrix 

[[1.0, 0.9994612499306779, 0.9989083421536935, 0.9983723118285689, 0.9978288491985361, 0.9972949462557793, 0.9967568455377309, 0.9962000482616908, 0.9956476385987072, 0.9950757022954487], [0.9994612499306779, 1.0, 0.9994612499306779, 0.9989083421536935, 0.9983723118285689, 0.9978288491985361, 0.9972949462557793, 0.9967568455377309, 0.9962000482616908, 0.9956476385987072], [0.9989083421536935, 0.9994612499306779, 1.0, 0.9994612499306779, 0.9989083421536935, 0.9983723118285689, 0.9978288491985361, 0.9972949462557793, 0.9967568455377309, 0.9962000482616908], [0.9983723118285689, 0.9989083421536935, 0.9994612499306779, 1.0, 0.9994612499306779, 0.9989083421536935, 0.9983723118285689, 0.9978288491985361, 0.9972949462557793, 0.9967568455377309], [0.9978288491985361, 0.9983723118285689, 0.9989083421536935, 0.9994612499306779, 1.0, 0.9994612499306779, 0.9989083421536935, 0.9983723118285689, 0.9978288491985361, 0.9972949462557793], [0.9972949462557793, 0.997828849198